In [6]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('Bengaluru_House_Data.csv')
df.sample(5)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
11387,Super built-up Area,Ready To Move,Kambipura,2 BHK,Prrtht,883,2.0,1.0,45.0
12695,Super built-up Area,Ready To Move,7th Phase JP Nagar,2 BHK,Prhna S,1128,2.0,1.0,60.0
7869,Plot Area,Ready To Move,Vaishnavi Layout,4 Bedroom,Prsisge,4800,5.0,2.0,420.0
1697,Plot Area,Ready To Move,Ramagondanahalli,1 Bedroom,NaN,540,1.0,0.0,30.0
4476,Super built-up Area,Ready To Move,7th Phase JP Nagar,2 BHK,BriumMi,1215,2.0,2.0,90.0


In [3]:
df.isnull().mean()*100

area_type        0.000000
availability     0.000000
location         0.007508
size             0.120120
society         41.306306
total_sqft       0.000000
bath             0.548048
balcony          4.572072
price            0.000000
dtype: float64

In [4]:
df.dtypes

area_type        object
availability     object
location         object
size             object
society          object
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

In [11]:
df['balcony']=df['balcony'].fillna(np.mean(df['balcony']))
df['bath']=df['bath'].fillna(np.mean(df['bath']))
df['total_sqft'].unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      shape=(2117,), dtype=object)

In [12]:
def convert_sqft(x):
    try:
        if '-' in x:
            parts = x.split('-')
            return (float(parts[0]) + float(parts[1])) / 2
        else:
            return float(x)
    except:
        return np.nan  # In case of unexpected bad data

# Apply function
df['total_sqft'] = df['total_sqft'].apply(convert_sqft)

In [17]:
df['total_sqft']=df['total_sqft'].fillna(np.mean(df['total_sqft']))
df.isnull().mean()*100

area_type        0.000000
availability     0.000000
location         0.007508
size             0.120120
society         41.306306
total_sqft       0.000000
bath             0.000000
balcony          0.000000
price            0.000000
dtype: float64

In [20]:
df.sample(5)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
10130,Built-up Area,Ready To Move,Jigani,3 BHK,DLhtsen,1252.0,3.0,2.000000,59.0
4290,Super built-up Area,Ready To Move,Hormavu,2 BHK,NStusD,1153.0,2.0,1.000000,65.0
4822,Plot Area,Ready To Move,Upkar Layout,7 Bedroom,NaN,1200.0,7.0,3.000000,169.0
9185,Plot Area,Ready To Move,Mico Layout,4 Bedroom,NaN,3600.0,4.0,1.584376,330.0
10303,Super built-up Area,18-May,Electronic City Phase II,2 BHK,MJtroty,972.0,2.0,1.000000,40.0


In [21]:
df.drop(columns=['size','society'],inplace=True)

In [25]:
df.describe()

,total_sqft,bath,balcony,price
count,13320.000000,13320.000000,13320.000000,13320.000000
mean,1559.626694,2.692610,1.584376,112.565627
std,1236.264860,1.337777,0.798360,148.971674
min,1.000000,1.000000,0.000000,8.000000
25%,1100.000000,2.000000,1.000000,50.000000
50%,1279.500000,2.000000,2.000000,72.000000
75%,1678.000000,3.000000,2.000000,120.000000
max,52272.000000,40.000000,3.000000,3600.000000


In [38]:
df=df[df['bath']<15]
df=df[df['total_sqft']<30000]
df=df[df['price']<2300]

In [39]:
df.describe()

,total_sqft,bath,balcony,price
count,13300.000000,13300.000000,13300.000000,13300.000000
mean,1538.205982,2.680944,1.584195,110.527229
std,938.232016,1.253614,0.798172,131.434317
min,1.000000,1.000000,0.000000,8.000000
25%,1100.000000,2.000000,1.000000,50.000000
50%,1278.000000,2.000000,2.000000,72.000000
75%,1674.000000,3.000000,2.000000,120.000000
max,26136.000000,14.000000,3.000000,2250.000000


In [ ]:
df['price'] = np.log1p(df['price'])


## Model Training

In [44]:
x=df.drop(columns='price')
y=df['price']
df.dtypes

area_type        object
availability     object
location         object
total_sqft      float64
bath            float64
balcony         float64
price           float64
dtype: object

In [46]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import optuna
import xgboost as xgb
from sklearn.metrics import r2_score

In [ ]:
categorical_cols = ['area_type', 'availability', 'location']
numerical_cols = ['total_sqft', 'bath', 'balcony']

preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols),
    ('num', StandardScaler(), numerical_cols)
])


In [49]:
def objective(trial):
    model_name = trial.suggest_categorical('model', ['LinearRegression', 'RandomForest', 'XGBoost'])
    test_size = trial.suggest_float('test_size', 0.1, 0.3)
    random_state = trial.suggest_int('random_state', 1, 100)

    X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=test_size, random_state=random_state)

    if model_name == 'LinearRegression':
        model = LinearRegression()
        
    elif model_name == 'RandomForest':
        n_estimators = trial.suggest_int('rf_n_estimators', 100, 500)
        max_depth = trial.suggest_int('rf_max_depth', 3, 20)
        min_samples_split = trial.suggest_int('rf_min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('rf_min_samples_leaf', 1, 5)
        model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=random_state,
            n_jobs=-1
        )

    elif model_name == 'XGBoost':
        n_estimators = trial.suggest_int('xgb_n_estimators', 100, 500)
        max_depth = trial.suggest_int('xgb_max_depth', 3, 15)
        learning_rate = trial.suggest_float('xgb_learning_rate', 0.01, 0.3)
        subsample = trial.suggest_float('xgb_subsample', 0.5, 1.0)
        colsample_bytree = trial.suggest_float('xgb_colsample_bytree', 0.5, 1.0)
        
        model = xgb.XGBRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            random_state=random_state,
            n_jobs=-1
        )

    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_valid)
    score=r2_score(y_valid,y_pred)
    return score

In [50]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100,show_progress_bar=True)

[I 2025-07-10 18:07:30,337] A new study created in memory with name: no-name-12f2b9fb-2dac-4a32-8c22-978c4122e6d5
Best trial: 0. Best value: 0.760051:   1%|          | 1/100 [00:06<09:58,  6.04s/it]

[I 2025-07-10 18:07:36,379] Trial 0 finished with value: 0.760051154918895 and parameters: {'model': 'RandomForest', 'test_size': 0.24519675941096514, 'random_state': 53, 'rf_n_estimators': 500, 'rf_max_depth': 12, 'rf_min_samples_split': 5, 'rf_min_samples_leaf': 3}. Best is trial 0 with value: 0.760051154918895.


Best trial: 1. Best value: 0.78139:   3%|▎         | 3/100 [00:12<05:28,  3.39s/it] 

[I 2025-07-10 18:07:42,583] Trial 1 finished with value: 0.7813901305188049 and parameters: {'model': 'RandomForest', 'test_size': 0.14082104874021403, 'random_state': 20, 'rf_n_estimators': 390, 'rf_max_depth': 14, 'rf_min_samples_split': 9, 'rf_min_samples_leaf': 4}. Best is trial 1 with value: 0.7813901305188049.
[I 2025-07-10 18:07:42,699] Trial 2 finished with value: 0.7415600045030983 and parameters: {'model': 'LinearRegression', 'test_size': 0.22350813059052074, 'random_state': 19}. Best is trial 1 with value: 0.7813901305188049.


Best trial: 3. Best value: 0.832156:   6%|▌         | 6/100 [00:13<01:46,  1.13s/it]

...

[I 2025-07-10 18:09:54,846] Trial 96 finished with value: 0.7988984761440631 and parameters: {'model': 'RandomForest', 'test_size': 0.12526865813707894, 'random_state': 30, 'rf_n_estimators': 422, 'rf_max_depth': 17, 'rf_min_samples_split': 2, 'rf_min_samples_leaf': 2}. Best is trial 93 with value: 0.8613817402823094.


Best trial: 93. Best value: 0.861382:  99%|█████████▉| 99/100 [02:26<00:02,  2.32s/it]

[I 2025-07-10 18:09:56,657] Trial 97 finished with value: 0.84443747509957 and parameters: {'model': 'XGBoost', 'test_size': 0.10893125769194115, 'random_state': 26, 'xgb_n_estimators': 473, 'xgb_max_depth': 15, 'xgb_learning_rate': 0.24283547933802024, 'xgb_subsample': 0.5756776231807568, 'xgb_colsample_bytree': 0.6637110643261503}. Best is trial 93 with value: 0.8613817402823094.
[I 2025-07-10 18:09:56,797] Trial 98 finished with value: 0.7473626922222221 and parameters: {'model': 'LinearRegression', 'test_size': 0.11859924861774454, 'random_state': 35}. Best is trial 93 with value: 0.8613817402823094.


Best trial: 93. Best value: 0.861382: 100%|██████████| 100/100 [02:28<00:00,  1.49s/it]

[I 2025-07-10 18:09:58,973] Trial 99 finished with value: 0.8198371854197608 and parameters: {'model': 'XGBoost', 'test_size': 0.13202470157109253, 'random_state': 39, 'xgb_n_estimators': 495, 'xgb_max_depth': 15, 'xgb_learning_rate': 0.19936666316322083, 'xgb_subsample': 0.6308693362517743, 'xgb_colsample_bytree': 0.7104328322922785}. Best is trial 93 with value: 0.8613817402823094.


In [51]:
best_params = study.best_trial.params
print("Best Hyperparameters:")
print(best_params)
print(f"Best R² Score: {study.best_value:.4f}")

Best Hyperparameters:
{'model': 'XGBoost', 'test_size': 0.1347662709076835, 'random_state': 30, 'xgb_n_estimators': 455, 'xgb_max_depth': 14, 'xgb_learning_rate': 0.18579052942455954, 'xgb_subsample': 0.5467891860641703, 'xgb_colsample_bytree': 0.7159168039483006}
Best R² Score: 0.8614


## Store the Model


In [57]:
import pickle

In [52]:
best_model_name = best_params['model']
best_random_state = best_params['random_state']
best_test_size = best_params['test_size']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=best_test_size, random_state=best_random_state)


if best_model_name == 'LinearRegression':
    best_model = LinearRegression()

elif best_model_name == 'RandomForest':
    best_model = RandomForestRegressor(
        n_estimators=best_params['rf_n_estimators'],
        max_depth=best_params['rf_max_depth'],
        min_samples_split=best_params['rf_min_samples_split'],
        min_samples_leaf=best_params['rf_min_samples_leaf'],
        random_state=best_params['random_state'],
        n_jobs=-1
    )

elif best_model_name == 'XGBoost':
    best_model = xgb.XGBRegressor(
        n_estimators=best_params['xgb_n_estimators'],
        max_depth=best_params['xgb_max_depth'],
        learning_rate=best_params['xgb_learning_rate'],
        subsample=best_params['xgb_subsample'],
        colsample_bytree=best_params['xgb_colsample_bytree'],
        random_state=best_params['random_state'],
        n_jobs=-1
    )

final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model)
])

final_pipeline.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [54]:
y_pred = final_pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"Test R² Score: {r2:.4f}")


Test R² Score: 0.8614


In [58]:
with open('best_model_pipeline.pkl', 'wb') as f:
    pickle.dump(final_pipeline, f)